# Module 09: Design and implement a replication strategy for Azure Cosmos DB SQL API

- [[Learning path]](https://docs.microsoft.com/en-us/learn/paths/design-implement-replication-strategy-cosmos-db-sql-api/?ns-enrollment-type=Collection&ns-enrollment-id=1k8wcz8zooj2nx)
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/20-sdk-regions.html): Connect to different regions with the Azure Cosmos DB SQL API SDK
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/21-sdk-consistency-model.html): Configure consistency models in the portal and the Azure Cosmos DB SQL API SDK
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/22-sdk-multi-region.html): Connect to a multi-region write account with the Azure Cosmos DB SQL API SDK

## Demo setup

In [ ]:
Connect-AzAccount
Set-AzContext -Subscription "b895a719-7034-411a-9944-ff196d1f450f"
$connectionString = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "ConnectionStrings")["Primary SQL Connection String"]
$primaryMasterKey = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "Keys")["PrimaryMasterKey"]
$documentEndpoint = (Get-AzCosmosDBAccount -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned).DocumentEndpoint

In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: Microsoft.Azure.Cosmos, 3.26.0"

#!share --from pwsh connectionString
#!share --from pwsh primaryMasterKey
#!share --from pwsh documentEndpoint

public class Product
{
	public string id { get; set; }
	public string categoryId { get; set; }
	public string categoryName { get; set; }
	public string sku { get; set; }
	public string name { get; set; }
	public string description { get; set; }
	public double price { get; set; }
}

## Configure replication and manage failovers in Azure Cosmos DB

### Understand replication

A replica set is a group of replicas that can dynamically grow and shrink to meet the needs of the database platform.

Each replica set will have other geographically distant replica sets that manage the same partition keys if data is distributed globally. These replica sets can then forward data to other replica sets in different regions to create replica copies of the data.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/2-replica-sets.png)

An Azure Cosmos DB account replicates data within a region (local distribution) among different replica sets servicing various partition key values. Replica sets that manage the same partition key value are referred to as a partition set as they will forward data between each other (global distribution).

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/2-partition-sets.png)

### Distribute data across regions

Configuring global distribution in Azure Cosmos DB is a turnkey operation that is performed when an account is created or afterward.

Configuring geo-redundancy for a new account:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/3-geo-redundancy.png)

Configuring geo-redundancy for an existing account:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/3-replication-map.png)

The cost of distributing data globally is **RU/s x # of regions**.

For example, consider a solution that uses approximately 1,000 RU/s per hour; data is written to one Azure region and replicated to five more regions. The formula for this would be:

```bash
1,000 x (1+5) = 6,000
```

The account would be billed for **6,000 RU/s** used at a per-hour rate.

### Azure Cosmos DB failovers

An automatic failover plan can transfer the write region to one of the read regions in the case of an outage.

Define automatic failover policies:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/5-automatic-failover.png)

Perform manual failovers:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-replication-manage-failovers-azure-cosmos-db/media/6-manual-failover.png)

### Configure SDK region

Use the ApplicationRegion or ApplicationPreferredRegions properties to configure preferred regions.

In [ ]:
// Setting an application region

using Microsoft.Azure.Cosmos;

CosmosClientOptions options = new () { ApplicationRegion = Regions.UKSouth }; 
CosmosClient client = new (connectionString, options);

In [ ]:
//  Or using the CosmosClientBuilder class

using Microsoft.Azure.Cosmos.Fluent;

CosmosClient client = new CosmosClientBuilder(connectionString) 
    .WithApplicationRegion(Regions.UKSouth) 
    .Build();

In [ ]:
// Setting a list of preferred application regions

List<string> regions = new() { "East Asia", "South Africa North", "West US" }; 
CosmosClientOptions options = new () { ApplicationPreferredRegions = regions }; 
CosmosClient client = new (connectionString, options);

In [ ]:
//  Or using the CosmosClientBuilder class

CosmosClient client = new CosmosClientBuilder(connectionString) 
  .WithApplicationPreferredRegions( new List<string> 
    { 
        Regions.EastAsia, 
        Regions.SouthAfricaNorth, 
        Regions.WestUS 
    } ) .Build();

## Use consistency models in Azure Cosmos DB SQL API

### Understand consistency models

In a distributed database system, tradeoffs are often made between highly consistent data with extended latency and speedy data operations that may not be consistent immediately.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/use-consistency-models-azure-cosmos-db-sql-api/media/2-sliding-scale.png)

Each of the five consistency levels is well-defined with clear tradeoffs when compared with each other:

| **Consistency Level** | **Description** |
| ---: | --- |
| **Strong** | Linear consistency. Data is replicated and committed in all configured regions before acknowledged as committed and visible to all clients. |
| **Bounded Staleness** | Reads lag behind writes by a configured threshold in time or items. |
| **Session** | Within a specific session (SDK instance), users can read their own writes. |
| **Consistent Prefix** | Reads may lag behind writes, but reads will never appear out of order. |
| **Eventual** | Reads will eventually be consistent with writes. |

### Configure default consistency model in the portal

In the Azure portal, the Default consistency pane is used to configure a new default consistency level for the entire account.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/use-consistency-models-azure-cosmos-db-sql-api/media/3-default-consistency.png)

### Change consistency model with the SDK

Using the **ItemRequestOptions** class, you can relax the current default consistency level to a weaker one.

In [ ]:
// Set the item request Consistency Level option
ItemRequestOptions options = new() 
{ 
    ConsistencyLevel = ConsistencyLevel.Eventual 
};

// Assign the option to the create item operation
var item = new Product 
{ 
    id = $"{Guid.NewGuid()}", 
    categoryId = $"{Guid.NewGuid()}", 
    name = "Reflective Handlebar"
}; 

CosmosClient client = new (connectionString);
Database database = await client.CreateDatabaseIfNotExistsAsync("cosmicworks");
Container container = await database.CreateContainerAsync("cosmicworks", "/categoryId", 400 );

await container.CreateItemAsync<Product>(item, requestOptions: options);

### Use session tokens

Using the .NET SDK classes, the session token can be manually extracted and passed back to the Azure Cosmos DB resource.

In [ ]:
// Create an item with an Item Response
var id = $"{Guid.NewGuid()}";
var categoryId = $"{Guid.NewGuid()}";
var item = new Product 
{ 
    id = id, 
    categoryId = categoryId, 
    name = "Reflective Handlebar 2"
}; 
ItemResponse<Product> response = await container.CreateItemAsync<Product>(item); 

// Get the session token from the item response
string token = response.Headers.Session;

// Set the item request option session token to the previous token
ItemRequestOptions options = new() 
{ 
    SessionToken = token 
}; 

// Use the token on the new request
ItemResponse<Product> readResponse = await container.ReadItemAsync<Product>(
    id, new PartitionKey(categoryId), requestOptions: options);

Console.WriteLine($"Session token: {token}");

## Configure multi-region write in Azure Cosmos DB SQL API

### Understand multi-region write

With Azure Cosmos DB, every region supports both writes and reads. 

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-multi-region-write-azure-cosmos-db-sql-api/media/2-enable-multi-write.png)

### Understand conflict resolution policies

Azure Cosmos DB’s multi-region write feature has automatic conflict management built in. The default policy is known as Last Write Wins that uses the _ts property by default.

Replace the default _ts property configuring any numeric property as a conflict resolution path on the .NET SDK

In [ ]:
Database database = client.GetDatabase("cosmicworks"); 

// Define a default custom conflict resolution path as /metadata/sortableTimestamp.
ContainerProperties properties = new("products", "/categoryId") 
{ 
    ConflictResolutionPolicy = new ConflictResolutionPolicy() 
    { 
        Mode = ConflictResolutionMode.LastWriterWins, 
        ResolutionPath = "/sortableTimestamp"  // "/metadata/sortableTimestamp"  will return Invalid path '\/metadata\/sortableTimestamp' for last writer wins conflict resolution
    } 
}; 

// Note: You can only set a conflict resolution policy on newly created containers.
//await container.DeleteContainerAsync();
container = await database.CreateContainerIfNotExistsAsync(properties);

### Create a custom conflict resolution policy step 1: JavaScript SP

Create a custom conflict resolution policy when you wish to write your own logic to resolve conflicts between items. Let’s first define the custom JavaScript SP that resolves the conflicts.

```javascript
function <function-name>(incomingItem, existingItem, isTombstone, conflictingItems)
```

| **Parameter** | **Description** |
| --- | :--- |
| **existingItem** | The item that is already committed |
| **incomingItem** | The item that's being inserted or updated that generated the conflict |
| **isTombstone** | Boolean indicating if the incoming item was previously deleted |
| **conflictingItems** | Array of all committed items in the container that conflicts with incomingItem |

This sample stored procedure resolves conflicts by selecting the lowest value from the /myCustomId path.

```javascript
function resolver(incomingItem, existingItem, isTombstone, conflictingItems) {
  var collection = getContext().getCollection();

  if (!incomingItem) {
      if (existingItem) {

          collection.deleteDocument(existingItem._self, {}, function (err, responseOptions) {
              if (err) throw err;
          });
      }
  } else if (isTombstone) {
      // delete always wins.
  } else {
      if (existingItem) {
          if (incomingItem.myCustomId > existingItem.myCustomId) {
              return; // existing item wins
          }
      }

      var i;
      for (i = 0; i < conflictingItems.length; i++) {
          if (incomingItem.myCustomId > conflictingItems[i].myCustomId) {
              return; // existing conflict item wins
          }
      }

      // incoming item wins - clear conflicts and replace existing with incoming.
      tryDelete(conflictingItems, incomingItem, existingItem);
  }

  function tryDelete(documents, incoming, existing) {
      if (documents.length > 0) {
          collection.deleteDocument(documents[0]._self, {}, function (err, responseOptions) {
              if (err) throw err;

              documents.shift();
              tryDelete(documents, incoming, existing);
          });
      } else if (existing) {
          collection.replaceDocument(existing._self, incoming,
              function (err, documentCreated) {
                  if (err) throw err;
              });
      } else {
          collection.createDocument(collection.getSelfLink(), incoming,
              function (err, documentCreated) {
                  if (err) throw err;
              });
      }
  }
}
```

### Create a custom conflict resolution policy step 2: .NET SDK

Once the JavaScript SP code has been defined, let’s assume it was written to the resolver.js file.

In [ ]:
using Microsoft.Azure.Cosmos.Scripts;

Database database = client.GetDatabase("cosmicworks"); 

// Define the custom conflict resolution path as /metadata/sortableTimestamp.
ContainerProperties properties = new("productswithconflict", "/categoryId") 
{ 
    ConflictResolutionPolicy = new ConflictResolutionPolicy() 
    { 
        Mode = ConflictResolutionMode.Custom, 
        ResolutionProcedure = string.Format("dbs/{0}/colls/{1}/sprocs/{2}",
                                                "cosmicworks",
                                                "productswithconflict",
                                                "resolver") 
 
    } 
}; 

// Note: You can only set a conflict resolution policy on newly created containers.
Container container = await database.CreateContainerIfNotExistsAsync(properties);

await container.Scripts.CreateStoredProcedureAsync( 
    new StoredProcedureProperties("resolver", "function resolver(incomingItem, existingItem, isTombstone, conflictingItems) {\n    var collection = getContext().getCollection();\n\n    if (!incomingItem) {\n        if (existingItem) {\n\n            collection.deleteDocument(existingItem._self, {}, function (err, responseOptions) {\n                if (err) throw err;\n            });\n        }\n    } else if (isTombstone) {\n        // delete always wins.\n    } else {\n        if (existingItem) {\n            if (incomingItem.myCustomId > existingItem.myCustomId) {\n                return; // existing item wins\n            }\n        }\n\n        var i;\n        for (i = 0; i < conflictingItems.length; i++) {\n            if (incomingItem.myCustomId > conflictingItems[i].myCustomId) {\n                return; // existing conflict item wins\n            }\n        }\n\n        // incoming item wins - clear conflicts and replace existing with incoming.\n        tryDelete(conflictingItems, incomingItem, existingItem);\n    }\n\n    function tryDelete(documents, incoming, existing) {\n        if (documents.length > 0) {\n            collection.deleteDocument(documents[0]._self, {}, function (err, responseOptions) {\n                if (err) throw err;\n\n                documents.shift();\n                tryDelete(documents, incoming, existing);\n            });\n        } else if (existing) {\n            collection.replaceDocument(existing._self, incoming,\n                function (err, documentCreated) {\n                    if (err) throw err;\n                });\n        } else {\n            collection.createDocument(collection.getSelfLink(), incoming,\n                function (err, documentCreated) {\n                    if (err) throw err;\n                });\n        }\n    }\n}") 
);


### Create a custom conflict resolution policy

These samples show how to set up a container with a custom conflict resolution policy. These conflicts show up in the **conflict feed**.

In [ ]:
Container container = await client.GetDatabase("cosmicworks")
    .CreateContainerIfNotExistsAsync(new ContainerProperties("productscustom", "/categoryId")
    {
        ConflictResolutionPolicy = new ConflictResolutionPolicy()
        {
            Mode = ConflictResolutionMode.Custom
        }
    });

container

## Demo teardown

In [ ]:
CosmosClient client = new (connectionString);
Database database = client.GetDatabase("cosmicworks");

await database.DeleteAsync();